In [1]:
import pandas as pd
from src.database import MongoDB

# Connect to MongoDB
db_name = 'clean_data'
collection_name = 'slack_cleaned_data'
connection_string = 'mongodb://localhost:27017/'

clean_db = MongoDB(db_name=db_name, collection_name=collection_name, connection_string=connection_string)



In [2]:
import torch
from torch.utils.data import Dataset

In [3]:
# clean_db.remove_duplicates(field='article_url', collection_name=collection_name)

In [4]:
# Load data into a pandas DataFrame
df = pd.DataFrame(list(clean_db.collection.find()))

In [5]:
df.head()

,_id,title,category,published_date,views,content,article_url,source
0,6656d918d2ac58683f96c67a,,መዝናኛ,"September 4, 2019",13,ለቀንዳም\nበሬ ቤት አትከልክለው\n መቼም በአማርኛ ቋንቋ ውስጥ በርካታ...,https://www.press.et/Ama/?p=17296,slack
1,6656d918d2ac58683f96c67b,ቃል ተከበረ,መዝናኛ,"September 8, 2019",73,ቃል ተከበረ ? ብዬ ጥያቄ ነገር አነሳሁ ። የምን ቃል ብላችሁ ከጠየቃችሁ...,https://www.press.et/Ama/?p=17498,slack
2,6656d918d2ac58683f96c67c,የማታ ስጦታ,መዝናኛ,"September 9, 2019",46,ዕድሜ ሲመሽ አካላዊ ብርታትን ስለማሳጣቱ አስረጂ አያሻም። እንደወትሮው ሰ...,https://www.press.et/Ama/?p=17529,slack
3,6656d918d2ac58683f96c67d,ቦርጭን በአራት ደቂቃ,መዝናኛ,"September 10, 2019",100,ቦርጭ ከአመጋገብ ጋር በተያያዘ በሆድ ውስጥ በሚከማች ስብ አማካኝነት የ...,https://www.press.et/Ama/?p=17589,slack
4,6656d918d2ac58683f96c67e,አመት በአሉ ይቀየር,መዝናኛ,"September 5, 2019",61,መስከረም ሊጠባ ነው፡፡ የእንቁጣጣሹ በአል አስገምግሞ መጥቷል፡፡ “ቡሄን...,https://www.press.et/Ama/?p=17337,slack


#### Convert your DataFrame into a Hugging Face Dataset:


In [6]:
from datasets import Dataset

# Given the `data` is the MongoDB data
# remove the unnecessary columns
# df = df.drop(columns=['_id', 'image_url', 'article_url', 'published_date', 'author', 'source', 'summary', 'content'])
df.head()

,_id,title,category,published_date,views,content,article_url,source
0,6656d918d2ac58683f96c67a,,መዝናኛ,"September 4, 2019",13,ለቀንዳም\nበሬ ቤት አትከልክለው\n መቼም በአማርኛ ቋንቋ ውስጥ በርካታ...,https://www.press.et/Ama/?p=17296,slack
1,6656d918d2ac58683f96c67b,ቃል ተከበረ,መዝናኛ,"September 8, 2019",73,ቃል ተከበረ ? ብዬ ጥያቄ ነገር አነሳሁ ። የምን ቃል ብላችሁ ከጠየቃችሁ...,https://www.press.et/Ama/?p=17498,slack
2,6656d918d2ac58683f96c67c,የማታ ስጦታ,መዝናኛ,"September 9, 2019",46,ዕድሜ ሲመሽ አካላዊ ብርታትን ስለማሳጣቱ አስረጂ አያሻም። እንደወትሮው ሰ...,https://www.press.et/Ama/?p=17529,slack
3,6656d918d2ac58683f96c67d,ቦርጭን በአራት ደቂቃ,መዝናኛ,"September 10, 2019",100,ቦርጭ ከአመጋገብ ጋር በተያያዘ በሆድ ውስጥ በሚከማች ስብ አማካኝነት የ...,https://www.press.et/Ama/?p=17589,slack
4,6656d918d2ac58683f96c67e,አመት በአሉ ይቀየር,መዝናኛ,"September 5, 2019",61,መስከረም ሊጠባ ነው፡፡ የእንቁጣጣሹ በአል አስገምግሞ መጥቷል፡፡ “ቡሄን...,https://www.press.et/Ama/?p=17337,slack


In [7]:
df_subset = df[['title', 'category']]

In [8]:
df_subset = df_subset.dropna()

In [9]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['title']
        labels = row['category']
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        encoding['labels'] = torch.tensor([labels])
        return encoding




# Assuming df is your DataFrame
# dataset = MyDataset(df_subset)

In [10]:
# Convert the DataFrame to a Hugging Face Dataset
# dataset = Dataset.from_pandas(df_subset)

In [11]:
# dataset = dataset.remove_columns(['__index_level_0__'])
# dataset = dataset.select(range(20))


#### Split the data into training and test set

In [12]:
# from torch.utils.data import random_split

# # Define the lengths of your train and test sets
# train_len = int(len(dataset) * 0.8)
# test_len = len(dataset) - train_len

# # Split the dataset
# train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

In [13]:
# Split the dataset into a train and test set
# dataset = dataset.train_test_split(test_size=0.2)
# dataset

In [14]:
# dataset['train']

## Load the models from hugging face

In [15]:
from transformers import (
    LlamaForCausalLM, 
    LlamaTokenizer, 
    TrainerCallback, 
    default_data_collator, 
    Trainer, 
    TrainingArguments
)

In [16]:
from peft import (
    LoraConfig,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training, # because prepare_model_for_int8_training is deprectated.
    PeftModel
)

#### Define the models

In [17]:
# Defin the Garri model name
model_name_llama2_7b = '/home/hillary_kipkemoi/hugging_face_models/garri/Llama-2-7b-hf'

# model_name_garri = '/home/hillary_kipkemoi/hugging_face_models/garri/llama-2-amharic-3784m/finetuned'

model_name_garri = "iocuydi/llama-2-amharic-3784m"
commit_hash = "04fcac974701f1dab0b8e39af9d3ecfce07b3773"


tokenizer = LlamaTokenizer.from_pretrained(model_name_garri, revision=commit_hash)

In [18]:
from sklearn.preprocessing import LabelEncoder


In [19]:
# Assuming df is your DataFrame
# tokenizer = LlamaTokenizer()  # Instantiate your custom tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add padding token

# Create a label encoder and fit it to your labels
label_encoder = LabelEncoder()
label_encoder.fit(df['category'])

dataset = MyDataset(df, tokenizer, label_encoder)

In [20]:
from torch.utils.data import random_split

# Define the lengths of your train and test sets
train_len = int(len(dataset) * 0.8)
test_len = len(dataset) - train_len

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

In [21]:



# TODO - preprocess by tokenizing on the train dataset
model = LlamaForCausalLM.from_pretrained(
    model_name_llama2_7b, 
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
    # cache_dir= cache_dir, # optional
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### Prepare the model

In [22]:
# Prepare the model for kbit training (helps reduce memory usage)
model = prepare_model_for_kbit_training(model)

#### Define the Lora Configuration

In [23]:
# Define LoRA configuration
lora_config = LoraConfig(
    # task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)

In [24]:
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

## Train the model
##### Define the training arguments

In [25]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./results/logs",
    # remove_unused_columns=False,
    logging_steps=10,
)

#### Define the trainer

In [26]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=dataset['train'],
    train_dataset=train_dataset,
    # eval_dataset=dataset['test']
    eval_dataset=test_dataset
)



#### Actually the train the model

In [27]:
# Train the model
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


RuntimeError: expand(torch.cuda.FloatTensor{[8, 8, 1, 1, 512]}, size=[8, 1, 1, 512]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)